In [1]:
import os
from datetime import datetime
import shutil
from time import time
import numpy as np
from tqdm import tqdm

In [2]:
ERROR_MODE_TABLE = 'The table is not in the correct mode'

READ_MODE = ['r']

WRITE_MODE = ['w', 'a']

SCHEMAS = {'part': ['P_PARTKEY', 'P_NAME', 'P_MFGR',
                      'P_BRAND', 'P_TYPE', 'P_SIZE', 
                      'P_CONTAINER', 'P_RETAILPRICE', 'P_COMMENT'],
             'supplier': ['S_SUPPKEY', 'S_NAME', 'S_ADDRESS',
                          'S_NATIONKEY', 'S_PHONE', 'S_ACCTBAL',
                          'S_COMMENT'],
             'partsupp': ['PS_PARTKEY', 'PS_SUPPKEY', 'PS_AVAILQTY',
                          'PS_SUPPLYCOST', 'PS_COMMENT'],
             'customer': ['C_CUSTKEY', 'C_NAME', 'C_ADDRESS',
                          'C_NATIONKEY', 'C_PHONE', 'C_ACCTBAL',
                          'C_MKTSEGMENT', 'C_COMMENT'],
             'orders': ['O_ORDERKEY', 'O_CUSTKEY', 'O_ORDERSTATUS',
                        'O_TOTALPRICE', 'O_ORDERDATE', 'O_ORDERPRIORITY',
                        'O_CLERK', 'O_SHIPPRIORITY', 'O_COMMENT'],
             'lineitem': ['L_ORDERKEY', 'L_PARTKEY', 'L_SUPPKEY',
                          'L_LINENUMBER', 'L_QUANTITY', 'L_EXTENDERDPRICE',
                          'L_DISCOUNT', 'L_TAX', 'L_RETRUNFLAG',
                          'L_LINESTATUS', 'L_SHIPDATE', 'L_COMMITDATE',
                          'L_RECEIPTDATE', 'L_SHIPINSTRUCT', 'L_SHIPMODE',
                          'L_COMMENT'],
             'nation': ['N_NATIONKEY', 'N_NAME', 'N_REGIONKEY', 'N_COMMENT'],
             'region': ['R_REGIONKEY', 'R_NAME', 'R_COMMENT']}

MAX_LENGTH = 50000


In [3]:
class Table_COL:
    
    
    #------------------------
    #       UTILITIES
    #------------------------

    def open_table(self, mode):
        ''' Open the table and its index in the specified mode

        Inputs:
          mode (str) : the mode to open the file ('w', 'r', 'a', 'w+')
        '''

        self.file = open(self.path_file, mode)
        self.index = open(self.path_index, mode)
        self.mode = mode
        
    def close_table(self):
        ''' Close the table and its index and add remaning data to table
        '''

        # Before closing if the mode in write we append the data left in object
        if self.mode in WRITE_MODE:
            self.file.write(self.last_page_data)
            self.last_page_data = ''
        
        # And close everything
        self.file.close()
        self.index.close()
    
    def write(self, data, data_id):
        ''' Central fonction that write to table with the record/page paradigme

        Inputs: 
          data (str) : data to add to the table, should be :
                       'field|field|...|field@field|...|field@'
          data_id (str) : the unique id of the data that will be put in the index
                          if None, no index is added
        ''' 
        # Verify that the table is in correct mode
        assert self.mode in WRITE_MODE, ERROR_MODE_TABLE

        len_data = len(data)        
        # If there is more space push the data onto the page
        if self.last_page_len + len_data < PAGE_LENGTH:
            self.last_page_data += data
            self.last_page_len += len_data

        # Else append the data to the file
        else:
            self.file.write(self.last_page_data + '\n')
            self.last_page_data = data
            self.last_page_len = len_data
            self.page_num += 1
            self.offset = 0
        
        if data_id:
            # Write the index of the file
            self.index.write(f'{data_id}|{self.page_num}|{self.offset}\n')
            self.offset += 1

    def get_by_id(self, record_id):
        ''' Retrun a record based on its id using the index

        Input:
          record_id (str): the unique id of the record
        
        Output:
          (Table) : the record
        '''

        # Open the table and create output
        self.open_table('r')
        output = Table('output', '', self.fields, new_file=True)
        output.open_table('a')
        
        # Transorm the id into a string as it is the format of the read
        record_id_str = str(record_id) + '|'
        record_id_size = len(record_id_str)
                                      
        # Compare the first characters of each line of the index with the id
        for line in self.index:
            if line[:record_id_size] == record_id_str:
                line = line.split('|')
                page_id, offset = int(line[1]), int(line[2])

        # Read file until the right page and get the record with the offset
        for i, line in enumerate(self.file):
            if i == page_id:
                result = line.strip().split('@')[offset]
                output.write(result + '@', result.split('|')[0])
                break
                
        # Close the tables
        output.close_table()
        self.close_table()
        
        return output
    
    def print_table(self, number_pages=1):
        ''' Print the table beautifully to 'output_nice.txt'

        Input:
          number_pages (int): the number of the table pages that should be printed
        '''
        self.open_table('r')
        self.file.readline()
        
        list_max_size = [len(field)+4 for field in self.fields]
        
        for i, page in enumerate(self.file):
            if i < number_pages:
                results = [record.split('|') for record in page.split('@')[:-1]]
                tmp_lst_max_size = [max([len(record[i]) for record in results])+4 for i in range(len(self.fields))]
                list_max_size = [max(maxi, maxi_tmp) for (maxi, maxi_tmp) in zip(list_max_size, tmp_lst_max_size)]
            else:
                break
                
        self.close_table()
        self.open_table('r')
        
        with open('output_nice.txt', 'w') as nice:
            
            str_schema = [f'{value:^{offset}}' for value, offset in zip(self.fields, list_max_size)]
            nice.write(' ' + '│ '.join(str_schema))
            nice.write('\n─' + '┼─'.join(['─'*field_len for field_len in list_max_size]) + '\n')
            
            for i, page in enumerate(self.file):
                if i < number_pages:
                    line = [[f'{value:<{offset}}' for value, offset in zip(record.split('|'), list_max_size)]
                                                                                for record in page.split('@')[:-1]]
                    nice.write('\n'.join([' ' + '│ '.join(record) for record in line]))
                else:
                     break

        self.close_table()
        
    def fields2id(self, list_field_name):
        '''Gives the position of field in the table schema

        Input:
            list_fields_name (lst [str]) : a list of fields name
        Ouptut:
            (lst [int]) : position of each fields
        '''
        list_field_id = []
        for name in list_field_name:
            i = 0
            while self.fields[i] != name:
                i += 1
            list_field_id.append(i)
            
        return list_field_id
    
    #------------------------
    #  AGGREGATION (SUM)
    #------------------------
    
    def aggregate_sum(self, fields_to_sum, output_name='output', output_path=''):
        
    '''
    Add the fields selected of each record from the table

        Inputs:
            fields_to_sum (lst [str]) : a list of field names to add (fields must be numerical)
            output_name (str) : the name of the output file
            output_path (str) : the path of the output file
        
        Output:
          (Table) : results for each field
    '''
        
        t0 = time()
        
        sum_fields = [0]*len(fields_to_sum)
        fields_id = self.fields2id(fields_to_sum)
        compt_columns = 0
        compt_field = 0
        
        self.open_table('r')
        
        schema = self.file.readline().strip()

        for page in self.file:
            
            columns = page.split('@')
            len_columns = len(columns)

            for i in range(len_columns):
                
                if columns[i] == '':
    
                    compt_columns += 1
                
                elif compt_columns in fields_id:
                    
                    column = columns[i].split('|')
                    if column[-1] !='\n':
                        column_int = list(map(float,column))
                    else:
                        column_int = list(map(float,column[:-1]))
                    
                    sum_fields[compt_field] += sum(column_int)
                    
                    if (i+1)<len_columns:
                        compt_columns += 1
                        compt_field += 1
                else:
                    
                    if (i+1)<len_columns:
                        compt_columns += 1
        
        self.close_table()
        output = Table_COL(output_name, output_path, fields_to_sum, new_file=True)
        output.open_table('a')
        output.write('@'.join([str(s) for s in sum_fields]) + '@', None)
        output.close_table()
        
        t1 = time()
        T = t1-t0
        
        print(f'The aggregation took {T:.4f}s')
        
        return output
    
    #---------------------------------------------------
    #  PROJECTION (WITHOUT TAKING CARE OF DUPLICATES)
    #---------------------------------------------------
        
    def projection(self, fields_project, output_name='output', output_path=''):
        
    '''
    Project on selected fields

        Inputs:
          field_project (lst [str]) : the fileld names to project
          output_name (str) : the name of the output file
          output_path (str) : the path of the output file

        Output:
          (Table) : containing all fields from the table with possible duplicates
    '''
        
        t0 = time()
        
        output = Table_COL(output_name, output_path, fields_project, new_file=True)
        
        project_fields = [0]*len(fields_project)
        fields_id = self.fields2id(fields_project)
        compt_columns = 0
        
        self.open_table('r')
        
        schema = self.file.readline().strip()
        
        output.open_table('a')
        
        for page in self.file:
            
            columns = page.split('@')
            len_columns = len(columns)
            
            for i in range(len_columns):
                
                if columns[i] == '':
                    compt_columns += 1
                
                elif compt_columns in fields_id:
                    column = columns[i].split('|')
                    
                    if column[-1] == '\n':
                         column = column[:-1]
                    
                    for data in column[:-1]:
                        output.write(data+'|', None)
                        
                    if (i+1)<len_columns:
                        compt_columns += 1
                        output.write(column[-1] + '@', None)
                else:
                    if (i+1)<len_columns:
                        compt_columns += 1
                        
        output.close_table()
        
        t1 = time()
        T = t1-t0
        
        print(f'The projection took {T:.4f}s')
        
        return output

### Tables (for testing)

In [4]:
path = 'datacolumns'

part_tbl = Table_COL('part', path)
lineitem_tbl = Table_COL('lineitem', path)

### Aggregation (sum) / COLUMNS

In [5]:
part_tbl.aggregate_sum(['P_PARTKEY', 'P_RETAILPRICE'])

The aggregation took 0.1864s


In [6]:
lineitem_tbl.aggregate_sum(['L_QUANTITY','L_EXTENDERDPRICE'])

The aggregation took 5.7043s


### Projection / COLUMNS

In [9]:
part_tbl.projection(['P_PARTKEY', 'P_NAME'])

The projection took 0.7765s


In [8]:
lineitem_tbl.projection(['L_ORDERKEY','L_PARTKEY'])

The projection took 21.5032s
